In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_FILE = "../../data/provision/expenses-report-data-configuration.json"

val userSdk = UserSdk()
val username = "Johann-12.0"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=5a724030-2d90-41b1-b776-520920d7fdd5, username=Johann-12.0)

In [3]:
import ro.jf.funds.fund.sdk.FundSdk

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, "Expenses") }
expenseFund

FundTO(id=8e952d91-0efb-4b52-9ae4-798a00b9bc70, name=Expenses)

In [4]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant
import kotlinx.serialization.json.Json

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }

    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_FILE).readText()
    val jsonFormat = Json { ignoreUnknownKeys = true }
    val dataConfiguration = jsonFormat.decodeFromString<ReportDataConfigurationTO>(dataConfigurationRawJson)

    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=f8983cc9-68c7-4b1b-b22d-492a42b77830, name=Expense report, fundId=8e952d91-0efb-4b52-9ae4-798a00b9bc70, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true))))

In [5]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=f8983cc9-68c7-4b1b-b22d-492a42b77830, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-12-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=DateInterval(from=2019-02-01, to=2019-02-28), net=-4301.32, value=ValueReportTO(start=2481.52, end=5185.08, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=DateInterval(from=2019-03-01, to=2019-03-31), net=-9171.45, value=ValueReportTO(start=5185.08, end=11301.26, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=DateInterval(from=2019-04-01, to=2019-04-30), net=-6967.76, value=ValueReportTO(start=11301.26, end=11526.0160226864, min=0.0, max=0.0)), ReportDataItemTO(timeBucket=DateInterval(from=2019-05-01, to=2019-05-31), net=-9043.57, value=ValueReportTO(start=11528.2727350348, end=10643.310981436, min=0.0, max=0.0)), ReportDataItemTO(

In [6]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    y("net")
    bars {
        x("month") {

        }
        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rVwHQx"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12],
"net":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"net",
"x":"month"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("rVwHQx");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 Jul 
 
 
 
 
 
 
 
 
 Aug 
 
 
 
 
 
 
 
 
 Sep 
 
 
 
 
 
 
 
 
 Oct 
 
 
 
 
 
 
 
 
 Nov 
 
 
 
 
 
 
 
 
 Dec 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 16,000 
 
 
 
 
 
 
 18,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 22,000 
 
 
 
 
 
 
 24,000 
 
 
 
 
 
 
 26,000 
 
 
 
 
 
 
 
 
 Monthly expenses 
 
 
 
 
 net 
 
 
 
 
 month

In [7]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MVxEss"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11528.2727350348,10643.764995302,11871.901585538,8638.3980216284,-6550.7530315744,-873.7461008179,4052.2818580708,6037.1321957254,2481.52,5185.08,11301.26,11526.0160226864,10643.310981436,11868.2374648476,8637.9027559568,-6550.8393119068,-872.2869408013,4051.7634872547,6037.1198616282,12394.3761693426]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("MVxEss");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M45.27221042399469 333.54513244175655 L45.27221042399469 333.54513244175655 L129.31044534278044 271.50634960214336 L205.2159478500762 203.91649671606365 L289.25418276886194 51.01006725831786 L370.5815068838201 45.33467726724365 L454.61974180260586 67.4476497942594 L535.9470659175568 36.74384760083194 L619.9853008363498 117.58245676477296 L704.0235357551355 497.3160247826532 L785.3508598700864 355.3890606009495 L869.3890947888794 232.23680762561222 L950.7164189038303 182.61492302791135 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(252,132,82)" stroke-opacity="1.0">
 
 
 
 <path d="M45.27221042399469 271.50634960214336 L45.27221042399469 271.50634960214336 L129.31044534278044 203.91649671606365 L205.2159478500762 51.01006725831786 L289.25418276886194 45.39109578787367 L370.5815068838201 67.45900028413621 L454.61974180260586 36.83545177400396 L535.9470659175568 117.59483856280332 L619.9853008363498 497.31818181818187 L704.0235357551355 355.3525811402165 L785.3508598700864 232.249767059544 L869.3890947888794 182.61523138423237 L950.7164189038303 23.681818181818244 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(84,112,198)" stroke-opacity="1.0">
 
 
 
 
 
 <